In [2]:
import numpy as np
import cv2
import os

In [64]:
vid = cv2.VideoCapture('VDO/left_output.avi')
template_img = cv2.imread('Template-1.png',1)
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)

min_match_number = 10
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(frame_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
        if len(good_matches) > min_match_number :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            src_pts,dst_pts = np.float32((src_pts,dst_pts))
            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
        cv2.imshow('Video frame', frame)
        if cv2.waitKey(33) & 0xFF == ord('q'): 
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()


In [8]:
AruCo_dict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_7X7_1000)
AruCo_params = cv2.aruco.DetectorParameters_create()
board = cv2.aruco.GridBoard_create(1, 1, 0.24, 0, AruCo_dict)

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv_contrib\modules\aruco\src\aruco.cpp:1510: error: (-215:Assertion failed) markersX > 0 && markersY > 0 && markerLength > 0 && markerSeparation > 0 in function 'cv::aruco::GridBoard::create'


In [5]:
def write_text(img, pose, dy, text) :
    x0 = pose[0]
    y0 = pose[1]
    for i, line in enumerate(text.split('\n')) :
        y = y0 + i*dy
        cv2.putText(img, line, np.int32([x0, y]), cv2.FONT_HERSHEY_COMPLEX, 0.75, (50,200,255), 2)

In [6]:
params_dir = os.getcwd()+'/new-camera_params/'

print(params_dir)

#load camera parameters
K = np.load(params_dir+'K.npy')
dist = np.load(params_dir+'dist.npy')

print("Camera matrix")
print(K)
print("Len distortion")
print(dist)

c:\Users\lingl\OneDrive\เอกสาร\GitHub\Image-processing.github.io/new-camera_params/
Camera matrix
[[1.31128729e+03 0.00000000e+00 6.30963210e+02]
 [0.00000000e+00 1.31095092e+03 4.75733608e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Len distortion
[[-0.39103383  0.09311993  0.00200492 -0.00161869  0.35166023]]


In [7]:
vid = cv2.VideoCapture('VDO/left_output.avi')
template_img = cv2.imread('Template-1.png',1)
template_gray = cv2.cvtColor(template_img, cv2.COLOR_BGR2GRAY)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 182)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 240)

min_match_number = 10
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

while vid.isOpened() :
    ret, frame = vid.read()
    
    if ret :
        img = frame.copy()
        markerCorners, markerIds, rejectedCandidates = cv2.aruco.detectMarkers(frame, AruCo_dict, parameters = AruCo_params)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        template_kpts, template_desc = sift.detectAndCompute(template_gray, None)
        query_kpts, query_desc = sift.detectAndCompute(frame_gray, None)
        matches = bf.knnMatch(template_desc, query_desc, k=2)
        good_matches = list()
        for m, n in matches :
            if m.distance < 0.7*n.distance :
                good_matches.append(m)
        if len(good_matches) > min_match_number :
            src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
            dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)
            src_pts,dst_pts = np.float32((src_pts,dst_pts))
            H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5) # H RANSAC
            # get the bounding box around template image
            h, w = template_img.shape[:2]
            template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
            transformed_box = cv2.perspectiveTransform(template_box, H)

            detected_img = cv2.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv2.LINE_AA)
        
        if len(markerCorners) > 0:
            frame = cv2.aruco.drawDetectedMarkers(frame, markerCorners)
            rvecs, tvecs, points = cv2.aruco.estimatePoseSingleMarkers(markerCorners , 0.05, K, dist)
            for (rvec, tvec, id, corner) in zip(rvecs, tvecs, markerIds, markerCorners) :
                frame = cv2.aruco.drawAxis(frame, K, dist, rvec, tvec, 0.05)
                x = tvec[0,0]
                y = tvec[0,1]
                z = tvec[0,2]
                text = "id: {}\n pose:\n {:.3f}\n {:.3f}\n {:.3f}".format(id, x, y, z)
                cX = (corner[0,0][0] + corner[0,2][0]) / 2
                cY = (corner[0,0][1] + corner[0,2][1]) / 2
                write_text(frame, (cX, cY), 20, text)
            ret, brvec, btvec = cv2.aruco.estimatePoseBoard(markerCorners, markerIds, board, K, dist, rvecs, tvecs) 
            if ret :
                frame = cv2.aruco.drawAxis(frame, K, dist, brvec, btvec, 0.05)

        cv2.imshow('Video frame', frame)
        if cv2.waitKey(33) & 0xFF == ord('q'): 
            break
    else :
        break
vid.release()
cv2.destroyAllWindows()